### Attribute Lookup chain review

In [1]:
class Child:
    name = "Liam"
    
    def __init__(self, name):
        self.name = name

In [2]:
Child.__dict__

mappingproxy({'__module__': '__main__',
              'name': 'Liam',
              '__init__': <function __main__.Child.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Child' objects>,
              '__weakref__': <attribute '__weakref__' of 'Child' objects>,
              '__doc__': None})

In [3]:
c = Child("Anthony")

In [4]:
c.__dict__

{'name': 'Anthony'}

In [5]:
c.name

'Anthony'

In [6]:
class Child:
    name = "Liam"
    
    def __init__(self, name=None):
        if name:
            self.name = name

In [7]:
c = Child()

In [9]:
c.__dict__

{}

In [8]:
c.name

'Liam'

In [22]:
class GrandParent:
    name = "Robert"

class Parent(GrandParent):
    # name = "James"
    pass

class Child(Parent):
    # name = "Liam"
    
    def __init__(self, name=None):
        if name:
            self.name = name

In [23]:
c = Child()

In [24]:
c.name

'Robert'

### Descriptor Protocol

In [ ]:
# protocols: contract between objects and py

# the descriptor protocol:
#     __get__()
#     __set__()
#     __delete__()

In [25]:
class Descriptor:
    def __get__(self, instance, owner):
        pass
    def __set__(self, instance, value):
        pass
    def __delete__(self, instance):
        pass

### Using a descriptor

In [2]:
# target: define a PersonTable class that has a first_name attribute that is text of max 200 len 
# class PersonTable: # something like this
#     first_name = TextField(200)

In [8]:
class TextField:
    def __init__(self, length) -> None:
        self.length = length
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        self.value = value
        
    def __delete__(self, instance):
        pass

In [9]:
class PersonTable:
    first_name = TextField(200)

In [10]:
p = PersonTable()

In [11]:
p.first_name = 'a' * 3

In [12]:
p.first_name

'aaa'

In [21]:
class TextField:
    def __init__(self, length) -> None:
        self.length = length
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        self.value = value
        
    def __delete__(self, instance):
        pass

In [22]:
class PersonTable:
    first_name = TextField(200)
    
    def __init__(self, first_name) -> None:
        self.__dict__["first_name"] = first_name

In [23]:
p = PersonTable("Robbie")

p.first_name = "Liam"

In [26]:
p.__dict__ # in instance dictionary it has robbie which should be priority but below it gives liam as descriptors are now priority before instance dict attributes

{'first_name': 'Robbie'}

In [27]:
p.first_name

'Liam'

### Descriptor Storage

In [4]:
class TextField:
    def __init__(self, length) -> None:
        self.length = length
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        self.value = value
        
    def __delete__(self, instance):
        pass
    
class PersonTable:
    first_name = TextField(200)

In [5]:
p1 = PersonTable()
p2 = PersonTable()

In [7]:
p1.first_name = "Andrew"

In [9]:
p2.first_name # changed for all the instances of persontable

'Andrew'

In [1]:
# way1
class TextField:
    def __init__(self, length) -> None:
        self.length = length
        self._data = {}
        
    def __get__(self, instance, owner):
        return self._data.get(instance, None)
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        # self.value = value
        self._data[instance] = value
        
    def __delete__(self, instance):
        pass
    
class PersonTable:
    first_name = TextField(200)

In [2]:
p1 = PersonTable()
p2 = PersonTable()

In [3]:
p1.first_name = "Andrew"

In [5]:
p2.first_name # now None

In [6]:
p2.first_name = "Bonnie"

In [7]:
p2.first_name

'Bonnie'

In [8]:
p1.first_name # different

'Andrew'

### Even better: Instance Storage

In [35]:
class TextField:
    def __init__(self, length) -> None:
        self.length = length
        self._data = {}
        
    def __get__(self, instance, owner):
        return instance.__dict__.get("text_field_value")
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        instance.__dict__["text_field_value"] = value
        
    def __delete__(self, instance):
        pass
    
class PersonTable:
    first_name = TextField(200)

In [13]:
p1  = PersonTable()
p2 = PersonTable()

In [14]:
p1.first_name = "Andrew"

In [17]:
p1.first_name, p2.first_name

('Andrew', None)

In [18]:
p2.first_name = "Bonnie"

In [19]:
p1.first_name, p2.first_name

('Andrew', 'Bonnie')

In [21]:
p1.__dict__, p2.__dict__

({'text_field_value': 'Andrew'}, {'text_field_value': 'Bonnie'})

In [36]:
# problem
class PersonTable:
    first_name = TextField(200)
    last_name = TextField(100)

In [37]:
p1 = PersonTable()

In [38]:
p1.first_name = "Andrew"
p1.last_name = "Green" 

In [39]:
p1.first_name, p1.last_name # both points to same because constant key name 

('Green', 'Green')

In [40]:
p1.__dict__

{'text_field_value': 'Green'}

In [28]:
# Solution

class TextField:
    def __init__(self, length, field_name) -> None:
        self.length = length
        self.field_name = field_name
        
    def __get__(self, instance, owner):
        return instance.__dict__.get(f"TextField_{self.field_name}")
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        instance.__dict__[f"TextField_{self.field_name}"] = value
        
    def __delete__(self, instance):
        pass

In [31]:
class PersonTable:
    first_name = TextField(200, "first_name")
    last_name = TextField(100, "last_name")

In [32]:
p1 = PersonTable()

p1.first_name = "Andrew"
p1.last_name = "Green" 

In [33]:
p1.first_name, p1.last_name

('Andrew', 'Green')

In [34]:
p1.__dict__

{'TextField_first_name': 'Andrew', 'TextField_last_name': 'Green'}

### Using __ set_name __

In [44]:
class TextField:
    def __init__(self, length) -> None:
        self.length = length
        
    def __set_name__(self, owner, name):
        # name is the name of class variable ("first_name, last_name etc")
        self.name = name
        
    def __get__(self, instance, owner):
        return instance.__dict__.get(f"TextField_{self.name}")
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        instance.__dict__[f"TextField_{self.name}"] = value
        
    def __delete__(self, instance):
        pass

In [46]:
class PersonTable:
    first_name = TextField(200) # because we have set_name dunder so we don't have overhead of giving field_name value
    last_name = TextField(100)

In [49]:
p1 = PersonTable()
p1.first_name = "Andrew" 
p1.last_name = "Green"

In [50]:
p1.first_name, p1.last_name

('Andrew', 'Green')

### Trying up loose ends

In [73]:
class TextField:
    def __init__(self, length) -> None: # self-> represents the instance of this class
        self.length = length
        
    def __set_name__(self, owner, name): # owner referes to the class that owns the descriptor (here persontable class is owner of this specific instances of descriptor(first_name, last_name))
        self.name = name
        
    def __get__(self, instance, owner): # instance: instance from which the descriptor is used (p1, p2..)
        print(instance)
        print(owner)
        if instance is None:
            return self
        return instance.__dict__.get(f"TextField_{self.name}")
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        instance.__dict__[f"TextField_{self.name}"] = value
        
    def __delete__(self, instance):
        del instance.__dict__[f"TextField_{self.name}"]

In [74]:
class PersonTable:
    first_name = TextField(200)
    last_name = TextField(100)

In [75]:
PersonTable.__dict__

mappingproxy({'__module__': '__main__',
              'first_name': <__main__.TextField at 0x25498b98b10>,
              'last_name': <__main__.TextField at 0x25498ab7c50>,
              '__dict__': <attribute '__dict__' of 'PersonTable' objects>,
              '__weakref__': <attribute '__weakref__' of 'PersonTable' objects>,
              '__doc__': None})

In [77]:
PersonTable.first_name

None
<class '__main__.PersonTable'>


In [78]:
p1 = PersonTable()
p1.first_name = "Andrew"

In [79]:
p1.first_name

<class '__main__.PersonTable'>


'Andrew'

In [80]:
del p1.first_name

In [81]:
p1.first_name

<class '__main__.PersonTable'>


### Non-data descriptor

In [2]:
class TextField:
    def __init__(self, length) -> None: # self-> represents the instance of this class
        self.length = length
        
    def __set_name__(self, owner, name): # owner referes to the class that owns the descriptor (here persontable class is owner of this specific instances of descriptor(first_name, last_name))
        self.name = name
        
    def __get__(self, instance, owner): # instance: instance from which the descriptor is used (p1, p2..)
        print(instance)
        print(owner)
        if instance is None:
            return self
        return instance.__dict__.get(f"TextField_{self.name}")
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        instance.__dict__[f"TextField_{self.name}"] = value
        
    def __delete__(self, instance):
        del instance.__dict__[f"TextField_{self.name}"]

In [1]:
from random import randint

class LuckyNumber:
    def __get__(self, instance, owner):
        return randint(1, 100)

In [7]:
class PersonTable:
    first_name = TextField(200)
    last_name = TextField(100)
    personal_no = LuckyNumber() # non-data descriptor
    
    def __init__(self, personal_no) -> None:
        self.personal_no = personal_no # instance attribute
    
p = PersonTable(personal_no=10000)

In [9]:
p.personal_no # here instance dict takes precedence over __get__ of non-data descriptor

10000

In [11]:
PersonTable.personal_no

97

In [12]:
from random import randint

class LuckyNumber: # make it as data descriptor
    def __get__(self, instance, owner):
        return randint(1, 100)
    
    def __set__(self, instance, value):
        pass

In [13]:
class PersonTable:
    first_name = TextField(200)
    last_name = TextField(100)
    personal_no = LuckyNumber() # data descriptor
    
    def __init__(self, personal_no) -> None:
        self.personal_no = personal_no # instance attribute
    
p = PersonTable(personal_no=10000)

In [14]:
p.personal_no # data descriptor takes precedence

98

### Descriptors vs Properties

In [15]:
class TextField:
    def __init__(self, length) -> None: # self-> represents the instance of this class
        self.length = length
        
    def __set_name__(self, owner, name): # owner referes to the class that owns the descriptor (here persontable class is owner of this specific instances of descriptor(first_name, last_name))
        self.name = name
        
    def __get__(self, instance, owner): # instance: instance from which the descriptor is used (p1, p2..)
        print(instance)
        print(owner)
        if instance is None:
            return self
        return instance.__dict__.get(f"TextField_{self.name}")
    
    def __set__(self, instance, value):
        if not type(value) == str:
            raise TypeError("Value should be a string")
        
        if len(value) > self.length:
            raise ValueError(f"Value cannot exceed {self.length} characters")
        
        instance.__dict__[f"TextField_{self.name}"] = value
        
    def __delete__(self, instance):
        del instance.__dict__[f"TextField_{self.name}"]
        
class PersonTableWithDescriptor:
    first_name = TextField(200)

In [17]:
class PersonTableWithProps: # same implementation with properties
    def __init__(self, first_name_length) -> None:
        self._TextField_first_name = None
        self.first_name_length = first_name_length
        
    @property
    def first_name(self):
        return self._TextField_first_name
    
    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")
        
        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name

In [18]:
p = PersonTableWithProps(200)

In [19]:
p.first_name = 2

TypeError: Value should be a string

In [23]:
p.first_name = "a" * 2000 # similar working as descriptors

ValueError: Value cannot exceed 200 characters

In [24]:
p.first_name = "Andrew" 

In [27]:
p.__dict__["first_name"] = "won't be reachable."

In [28]:
p.__dict__

{'_TextField_first_name': 'Andrew',
 'first_name_length': 200,
 'first_name': "won't be reachable."}

In [30]:
PersonTableWithProps.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.PersonTableWithProps.__init__(self, first_name_length) -> None>,
              'first_name': <property at 0x184ece99210>,
              '__dict__': <attribute '__dict__' of 'PersonTableWithProps' objects>,
              '__weakref__': <attribute '__weakref__' of 'PersonTableWithProps' objects>,
              '__doc__': None})

In [31]:
p.first_name # properties are just like data descriptors when it comes to attribute lookup precedence

'Andrew'

In [32]:
# add two new fields
# * last_name
# * occupation

In [33]:
class PersonTableWithProps:
    def __init__(self, first_name_length, last_name_length, occupation_length) -> None:
        self._TextField_first_name = None
        self._TextField_last_name = None
        self._TextField_occupation = None
        
        
        self.first_name_length = first_name_length
        self.last_name_length = last_name_length
        self.occupation_length = occupation_length
        
    @property
    def first_name(self):
        return self._TextField_first_name
    
    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")
        
        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name
        
    @property
    def first_name(self): # change it to last_name
        return self._TextField_first_name
    
    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")
        
        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name
        
    @property
    def first_name(self): # change it to occupation
        return self._TextField_first_name
    
    @first_name.setter
    def first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")
        
        self._TextField_first_name = value
    
    @first_name.deleter
    def first_name(self):
        del self._TextField_first_name # huge changes for 3 fields

In [35]:
# while descriptors has only 2 lines

class PersonTableWithDescriptor:# scalable
    first_name = TextField(200)
    last_name = TextField(200)
    occupation = TextField(100)

### Similarity

In [37]:
class PersonTableWithProps:
    def __init__(self, first_name_length) -> None:
        self._TextField_first_name = None
        self.first_name_length = first_name_length
    
    def get_first_name(self):
        return self._TextField_first_name
    
    def set_first_name(self, value):
        if not type(value) == str:
            raise TypeError(f"Value should be a string")

        if len(value) > self.first_name_length:
            raise ValueError(f"Value cannot exceed {self.first_name_length} characters")
        
        self._TextField_first_name = value
    
    def del_first_name(self):
        del self._TextField_first_name
        
    first_name = property(fget=get_first_name, fset=set_first_name, fdel=del_first_name) # similar syntax as descriptor below

In [38]:
class PersonTable:
    first_name = TextField(200)

### Skill Challenge 11

In [48]:
class SATScore:
    def __init__(self, score=400):
        self.score = score
        
    def __set_name__(self, owner, name):
        self.name = name
        
    def __get__(self, instance, owner):
        return instance.__dict__[f"SATScore_{self.name}"]
    
    def __set__(self, instance, value):
        if not type(value) == int:
            raise TypeError("Score should be int only!")
        
        if not 400 <= value <= 1600:
            raise ValueError("Invalid score. Score should be in range of 400 to 1600 only")
        
        instance.__dict__[f"SATScore_{self.name}"] = value

In [49]:
class GREScore:
    def __init__(self, score=130):
        self.score = score
        
    def __set_name__(self, owner, name):
        self.name = name
        
    def __get__(self, instance, owner): # if descriptor is accessed from a class. instance is none. so check that.
        if instance is None:
            return self
        
        return instance.__dict__[f"GREScore_{self.name}"]
    
    def __set__(self, instance, value):
        if not type(value) == int:
            raise TypeError("Score should be int only!")
        
        if not 130 <= value < 340:
            raise ValueError("Invalid score. Score should be in range of 130 to 340 only")
        
        instance.__dict__[f"GREScore_{self.name}"] = value

In [50]:
class StudentProfile:
    sat = SATScore()
    gre = GREScore()
    
    def __init__(self, name, sat=400, gre=130) -> None:
        self.name = name
        self.sat = sat # here the value comes to instance dict but as we have descriptor already named as sat so it takes precedence and __get__ is called in SATScore and sat is passed as value
        self.gre = gre
        
    def __repr__(self) -> str:
        return f"{type(self).__name__}(name='{self.name}', sat={self.sat}, gre={self.gre})"
    

In [51]:
sp = StudentProfile(name="Andrew", sat=1220, gre=131)

In [52]:
sp

StudentProfile(name='Andrew', sat=1220, gre=131)

In [53]:
sp.gre

131

In [54]:
sp.sat

1220

In [39]:
sp.sat = 200

ValueError: Invalid score. Score should be in range of 400 to 1600 only

In [40]:
sp.gre = 2.2

TypeError: Score should be int only!

In [41]:
sp.sat = "aab"

TypeError: Score should be int only!

### Only 1 descriptor

In [63]:
class ValidatedScore:
    def __init__(self, score=400, score_name=None, min_score=400, max_score=1600):
        self.score = score
        self.score_name = score_name
        self.min_score = min_score
        self.max_score = max_score
        
    def __set_name__(self, owner, name):
        self.name = name
        
    def __get__(self, instance, owner):
        return instance.__dict__[f"{self.score_name}_{self.name}"]
    
    def __set__(self, instance, value):
        if not type(value) == int:
            raise TypeError("Score should be int only!")
        
        if not self.min_score <= value <= self.max_score:
            raise ValueError(f"Invalid score. Score should be in range of {self.min_score} to {self.max_score} only")
        
        instance.__dict__[f"{self.score_name}_{self.name}"] = value

In [64]:
class SATScore(ValidatedScore):
    def __init__(self, score=400):
        super().__init__(score, score_name=self.__class__.__name__, min_score=400, max_score=1600)
        
class GREScore(ValidatedScore):
    def __init__(self, score=130):
        super().__init__(score, score_name=self.__class__.__name__, min_score=130, max_score=340)

In [65]:
class StudentProfile:
    sat = SATScore()
    gre = GREScore()
    
    def __init__(self, name, sat=400, gre=130) -> None:
        self.name = name
        self.sat = sat
        self.gre = gre
        
    def __repr__(self) -> str:
        return f"{type(self).__name__}(name='{self.name}', sat={self.sat}, gre={self.gre})"

In [66]:
sp = StudentProfile(name="Andrew", sat=1220, gre=131)

In [67]:
sp

StudentProfile(name='Andrew', sat=1220, gre=131)

In [68]:
sp.sat

1220

In [69]:
sp.gre

131

In [70]:
sp.gre = 120

ValueError: Invalid score. Score should be in range of 130 to 340 only

In [71]:
sp.sat = 12222

ValueError: Invalid score. Score should be in range of 400 to 1600 only

In [72]:
sp.__dict__

{'name': 'Andrew', 'SATScore_sat': 1220, 'GREScore_gre': 131}